In [4]:
#instalação de pacotes necessários
#install.packages("e1071")
#install.packages("caret")
#install.packages("mlbench")
#install.packages("mice")
#install.packages("Metrics")
library(caret)
library(mlbench)
library(mice)
library(Metrics)

In [5]:
#LEITURA DE DADOS

dados <- read.csv("databases/3 - Alunos - Dados.csv", header = TRUE)
head(dados)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,⋯,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,GP,2,18,U,GT3,A,4,4,at_home,teacher,⋯,4,3,4,1,1,3,6,5,6,6
2,GP,2,17,U,GT3,T,1,1,at_home,other,⋯,5,3,3,1,1,3,4,5,5,6
3,GP,2,15,U,LE3,T,1,1,at_home,other,⋯,4,3,2,2,3,3,10,7,8,10
4,GP,2,15,U,GT3,T,4,2,health,services,⋯,3,2,2,1,1,5,2,15,14,15
5,GP,2,16,U,GT3,T,3,3,other,other,⋯,4,3,2,1,2,5,4,6,10,10
6,GP,1,16,U,LE3,T,4,3,services,other,⋯,5,4,2,1,2,5,10,15,15,15


In [6]:
# cria arquivos de treino e teste

set.seed(1912)

indices <- createDataPartition(dados$G3, p = 0.80, list = FALSE)

treino <- dados[indices, ]
teste <- dados[-indices, ]

In [8]:
#treino com hold-out
set.seed(1912)
rna <- train(G3 ~ ., data = treino, method = "nnet",
             linout = TRUE, trace = FALSE)
rna
predict_rna <- predict(rna, teste)

Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
"There were missing values in resampled performance measures."


Neural Network 

318 samples
 32 predictor

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 318, 318, 318, 318, 318, 318, ... 
Resampling results across tuning parameters:

  size  decay  RMSE      Rsquared   MAE     
  1     0e+00  4.368180  0.2569692  3.257128
  1     1e-04  4.215986  0.3488204  3.159140
  1     1e-01  3.098748  0.5455339  2.441393
  3     0e+00  3.556612  0.4760443  2.739041
  3     1e-04  3.394274  0.5583555  2.573986
  3     1e-01  2.850064  0.6327496  2.164596
  5     0e+00  3.348612  0.5539436  2.489588
  5     1e-04  3.365850  0.5256625  2.555021
  5     1e-01  2.682002  0.6777950  1.968007

RMSE was used to select the optimal model using the smallest value.
The final values used for the model were size = 5 and decay = 0.1.

In [10]:
#calculo das metricas

rmse(teste$G3, predict_rna)

r2  <- function(predito, observado) {
  return(1 - (sum((predito - observado)^2) /
                sum((observado - mean(observado))^2)))
}
r2(predict_rna, teste$G3)

[1] 2.416473

[1] 0.7045448

In [9]:
#Cross validation e parametrização da RNA
ctrl <- trainControl(method = "cv", number = 10)
tune_grid <- expand.grid(size = seq(from = 1, to = 3, by = 1),
                         decay = seq(from = 0.1, to = 0.7, by = 0.3))
set.seed(1912)
rna <- train(G3 ~ ., data = treino, method = "nnet", trainControl = ctrl,
             tuneGrid = tune_grid, linout = TRUE, MaxNWts = 10000,
             maxit = 2000, trace = FALSE)
rna

Neural Network 

318 samples
 32 predictor

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 318, 318, 318, 318, 318, 318, ... 
Resampling results across tuning parameters:

  size  decay  RMSE      Rsquared   MAE     
  1     0.1    2.234037  0.7641828  1.592268
  1     0.4    2.270295  0.7547688  1.632889
  1     0.7    2.298572  0.7472927  1.667395
  2     0.1    2.537363  0.7121389  1.697008
  2     0.4    2.512691  0.7169540  1.672548
  2     0.7    2.527524  0.7071331  1.781073
  3     0.1    2.810359  0.6674822  1.979136
  3     0.4    2.368145  0.7425585  1.624389
  3     0.7    2.399160  0.7361881  1.711003

RMSE was used to select the optimal model using the smallest value.
The final values used for the model were size = 1 and decay = 0.1.

In [11]:
predict_rna <- predict(rna, teste)

rmse(teste$G3, predict_rna)
r2(predict_rna, teste$G3)

[1] 2.201599

[1] 0.7547526

In [12]:
# predicao de novos casos
dados_novos_casos <- read.csv("databases/3 - Alunos - Novos Casos.csv")
View(dados_novos_casos)

dados_novos_casos$G3 <- NULL
predict_rna <- predict(rna, dados_novos_casos)
resultado <- cbind(dados_novos_casos, predict_rna)
View(resultado)

school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,⋯,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
<chr>,<int>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>
GP,2,16,R,GT3,A,4,4,at_home,teacher,⋯,4,3,1,1,1,3,6,2,3,?
GP,1,17,U,GT3,T,1,1,at_home,other,⋯,5,3,3,1,1,3,4,15,15,?
GP,1,18,U,LE3,T,1,1,at_home,other,⋯,4,3,2,2,3,3,10,10,10,?


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,⋯,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,predict_rna
,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
1,GP,2,16,R,GT3,A,4,4,at_home,teacher,⋯,4,3,1,1,1,3,6,2,3,2.269679
2,GP,1,17,U,GT3,T,1,1,at_home,other,⋯,5,3,3,1,1,3,4,15,15,15.360997
3,GP,1,18,U,LE3,T,1,1,at_home,other,⋯,4,3,2,2,3,3,10,10,10,11.117258
